In [1]:
from importlib import reload
import flymovie as fm
from flymovie.simnuc import Sim
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import pickle
from pathlib import Path
import scipy.ndimage as ndi
from importlib import reload
import scipy
import pandas as pd
import scipy
from copy import deepcopy

%load_ext autoreload
%autoreload 2

In [57]:
stack = fm.read_czi('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/psf/2022-02-09/tetraspeck-field1-100x50.czi', swapaxes=False)
stack = stack + 0.5
mask =np.where(stack > 40, 1, 0)
#mask = ndi.morphology.binary_closing(mask, structure=np.ones((3,3,3)))
mask = ndi.morphology.binary_opening(mask, structure=np.ones((3,3,3)))
fm.viewer(mask, 6)

interactive(children=(Dropdown(description='Color', index=1, options=('Reds', 'cividis', 'gray', 'Greens', 'in…

In [142]:
from skimage.measure import regionprops
from flymovie.general_functions import extract_box, stack_normalize_minmax

def psf_from_60mer(stack, mask, box_dims, mode='centroid'):
    def out_of_bounds(centroid, stack_shape, box_dims):
        for d in range(len(box_dims)):
            halflen = int(box_dims[d] / 2)
            start = centroid[d] - halflen
            end = centroid[d] + halflen
            if start < 0:
                return True
            if end > (stack_shape[d]-1):
                return True
        return False

    psf = np.ndarray(tuple([0]) + box_dims)
    lmask,_ = ndi.label(mask)

    if mode == 'centroid':
        props = regionprops(lmask)
        centers = []
        for id_ in range(0, len(props)):
            centers.append(props[id_].centroid)

    elif mode == 'com':
        centers = ndi.measurements.center_of_mass(stack, lmask, np.arange(1, np.max(lmask) + 1))

    for center in centers:
        center = [round(x) for x in center]
        if out_of_bounds(center, stack.shape, box_dims):
            continue
        box = extract_box(stack, center, box_dims, pad=False)
        box = stack_normalize_minmax(box)
        box = np.expand_dims(box, axis=0)
        psf = np.vstack([psf, box])
        
    return psf


boxes_com = psf_from_60mer(stack, mask, (21,31,31), mode='com')

In [143]:
fm.viewer(boxes_com.max(axis=1), 5)

interactive(children=(Dropdown(description='Color', index=1, options=('Reds', 'cividis', 'gray', 'Greens', 'in…

In [145]:
bad=[29,31,32,36,37,38,42,43,44]

In [152]:
def remove_bad(stack, bad_indexes):
    mask = np.ones(stack.shape[0])
    mask[bad_indexes] = 0
    mask = mask.astype(bool)
    return(stack[mask])
good = remove_bad(boxes_com, bad)

In [158]:
fm.viewer(good.max(axis=(0,3))*1000, 5)

interactive(children=(Dropdown(description='Color', index=1, options=('Reds', 'cividis', 'gray', 'Greens', 'in…

In [159]:
sm = ndi.median_filter(good, 2)
fm.viewer(sm.max(axis=(0,3))*1000, 5)

interactive(children=(Dropdown(description='Color', index=1, options=('Reds', 'cividis', 'gray', 'Greens', 'in…

In [5]:
outfolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/batch_test'
fm.sim_rpb1_batch(outfolder, kernel_real, 2,1,1,(200,200,200), nuc_rad=92, nfree_rng=[10_000,50_000], hlb_diam_rng=[7,15], 
    hlb_nmols_rng=[100,1000], n_clusters_rng=[0,300], cluster_diam_mean_rng=[1,3], 
    cluster_diam_var_rng=[0.5, 2], cluster_nmols_mean_rng=[10,100], cluster_nmols_var_rng=[1,10], noise_sigma_rng=[1_000, 2_000])


arglist done


In [80]:
def rebin(arr, new_shape):
    """Bin a 2D numpy array to fit a new shape, with each entry
    in the binned array equal to the sum of the constituent
    entries in the original."""
    shape = ()
    for i in range(arr.ndim):
        shape = shape + (new_shape[i], arr.shape[i] // new_shape[i],)

    binned = arr.reshape(shape)
    for i in range(arr.ndim):
        binned = binned.sum(i + 1)
    
    return binned

## Tuning to real data

In [3]:
def norm(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

In [4]:
realstack = fm.read_czi('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/counting_blobs/cycle 12/embryo17.czi', swapaxes=False)

In [5]:
sub = norm(realstack[:,350:, 100:300]) * 1000
sub = sub[:,0:100, 25:125]

In [12]:
mask = Sim.make_spherical_mask(100,100,100, 40)
sim = Sim(mask, 85, 85)
sim.add_kernel(kernel_real, 250, 85)
# Free
sim.add_n_objects(10_000, 100, 1, 1, 'nuc')
# HLB
sim.add_sphere((47,35,35), 100, 400, 5)
sim.add_sphere((47,50,68), 100, 400, 5)
# clusters
sim.add_n_objects(60, 100, 10, 2)
#sim.add_n_objects(200, 100, 4, 2)
sim.add_noise('poisson')
sim.convolve()
sim.resize((250,85,85), order=1)
sim.im = norm(sim.im) * 1000
sim.add_noise('gaussian', sigma=20)
fm.viewer([sim.im * (np.mean(sub)/np.mean(sim.im)), sub], 5)


interactive(children=(Dropdown(description='Color', index=1, options=('Reds', 'cividis', 'gray', 'Greens', 'in…

In [9]:
np.min(fm.stack_normalize_minmax(sim.im))

0.0

In [11]:
np.random.random((20,20))

array([[0.58885976, 0.80369097, 0.83154375, 0.93872689, 0.38503927,
        0.82839788, 0.564153  , 0.64396203, 0.35173638, 0.72041955,
        0.52009986, 0.69649784, 0.92176532, 0.32224221, 0.7796964 ,
        0.21284628, 0.65489507, 0.28323664, 0.68993935, 0.24635628],
       [0.88567575, 0.96016432, 0.22195968, 0.96691403, 0.8362162 ,
        0.98081391, 0.1691706 , 0.07178698, 0.63351503, 0.13186   ,
        0.76064322, 0.18322916, 0.90451496, 0.71753499, 0.9172418 ,
        0.66567536, 0.81243367, 0.07514118, 0.27563948, 0.37396411],
       [0.19314588, 0.01142992, 0.3628084 , 0.58645793, 0.59270544,
        0.05922144, 0.94518293, 0.15507244, 0.87063839, 0.01532149,
        0.99358914, 0.74248937, 0.90590599, 0.2942996 , 0.93742622,
        0.08592347, 0.88411328, 0.29531802, 0.93380088, 0.86090825],
       [0.77650512, 0.02679758, 0.85563138, 0.87256157, 0.17839924,
        0.47910737, 0.16036844, 0.0243565 , 0.23537636, 0.01204458,
        0.3480154 , 0.40931393, 0.40908704, 0

In [23]:
np.mean(sub[0])

27.329019607843133

In [22]:
np.std(sub[0])

15.511245459892377

In [29]:
2 > 3 > 1

False